In [2]:
import gc
import os
import sys
import warnings

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import Binarizer, LabelEncoder, scale

warnings.filterwarnings('ignore')

In [3]:
data_path = '/home/kesci/input/smart_edu7557'
current_path = '/home/kesci/work'

dimension = 60
seed=7

In [4]:
# 简单统计
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)

    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()
    return df_merge

In [5]:
# cv 统计
def stat_cv(df, df_merge, group_by, on):
    group = df.groupby(group_by).agg({on: ['std', 'mean']})
    group.columns = ['std', 'mean']
    group.reset_index(inplace=True)

    group['cv'] = group['std'] / group['mean']
    group.drop(['std', 'mean'], axis=1, inplace=True)

    group.columns = group_by + ['{}_{}_cv'.format('_'.join(group_by), on)]
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()
    return df_merge

In [6]:
# rank 统计
def stat_rank(df, df_merge, group_by):
    df_temp = df.copy(deep=True)
    df_temp['rank'] = df.groupby(['exam_id'])['score'].rank(method='min',
                                                            ascending=False)

    df_temp = df_temp.groupby(group_by)['rank'].mean().to_frame()
    df_temp.reset_index(inplace=True)
    df_temp.columns = group_by + ['{}_rank_mean'.format('_'.join(group_by))]

    df_merge = df_merge.merge(df_temp, how='left')
    return df_merge

In [7]:
# student score mean / exam mean score
def stat_ratio_mean(df, df_merge, group_by):
    df_temp = df.copy(deep=True)
    df_temp['exam_mean_score'] = df.groupby(['exam_id'
                                             ])['score'].transform('mean')
    df_temp['student_score_ratio_exam_mean_score'] = df_temp[
        'score'] / df_temp['exam_mean_score']

    df_temp = df_temp.groupby(
        group_by)['student_score_ratio_exam_mean_score'].mean().to_frame()
    df_temp.reset_index(inplace=True)
    df_temp.columns = group_by + [
        'student_score_ratio_exam_mean_score_{}_mean'.format(
            '_'.join(group_by))
    ]

    df_merge = df_merge.merge(df_temp, how='left')
    return df_merge

In [8]:
def gen_feature(df, index, columns, values):
    p = df.pivot_table(index=index, columns=columns,
                       values=values).reset_index().fillna(0)
    columns = p.columns.values.tolist()
    columns = ['{}_{}'.format(c, values) if c != index else c for c in columns]
    p.columns = columns

    return p

In [9]:
def select_feature(df):
    p = 86

    df.fillna(0, inplace=True)
    ycol = 'score'
    feature_names = [f for f in df.columns if f not in [ycol, 'student_id', 'course', 'exam_id']]

    X = df[feature_names]
    Y = df[ycol]
    X_bin = Binarizer().fit_transform(scale(X))

    selectChi2 = SelectPercentile(chi2, percentile=p).fit(X_bin, Y)
    selectF_classif = SelectPercentile(f_classif, percentile=p).fit(X, Y)

    chi2_selected = selectChi2.get_support()
    print('Chi2 selected {} features.'.format(chi2_selected.sum()))
    f_classif_selected = selectF_classif.get_support()
    print('F_classif selected {} features.'.format(f_classif_selected.sum()))
    selected = chi2_selected & f_classif_selected
    print('Chi2 & F_classif selected {} features'.format(selected.sum()))
    features = [f for f, s in zip(feature_names, selected) if s]

    x = list(set(feature_names) - set(features))
    x.sort()
    print(x)
    df_sel = df[features]
    df_basic = df[[ycol, 'student_id', 'course', 'exam_id']]
    df_sel = pd.concat([df_sel, df_basic], axis=1)
    return df_sel

In [10]:
ids = ['t3gjIJ4S', 'XHQ4brE1', 'Dm8lzpUM', '7lQGsPpC', 'ldofMGhb', 'wUfEa8v2', 'YFZJ4rei', 
       '6gjiTmw7', '29N5PmSt', 'N43ekjOH', 'fkVo7mcd', '3TMeQo5F', 'aY0jKoh1', 'Bcn8WGsT', 
       'giCyVrdq', 'mHfYNU4q', 'aIoMYlV3', 'uQlfKROe', 'I5oCbFqU', '7OgaPmo8', 'vj28RUg0', 
       'woXyHNBr', '1tg3UJzG', '8YdaZ4eg', 'JHPfbgZ1', 'EZtGMeKm', 'IBREqrit', 'AsvqxgdU', 
       '9uAJinrI', 'Y1UQOByn', 'HcvGebr6', 'eSNJug3G', 'DkEa54bn', '7MuzF8Jo', 'Prp3YHoc', 
       'czAWLEuk', 'xSKstlN5', '8wUj4LGF', 'ZOyuld3U', 'YD843Ie1', 'rOsZ6xkQ', 'AFnBGQxE', 
       '2oStKrL7', 'SVANfGvI', '4CHUDKzQ', 'SkQCVrph', 'RFkVy6Gj', '4mLYGO6d', 'pNxEIa3i', 
       'k75ryC6E', 'rxoYgBcR', 'GzBKCNR0', 'Em0SdZCe', 'mKJgMDXv', 'tJEHCumY', 'k8GvsIxa', 
       'j8Tva0NC', 'sAKrHTBI', 'dOWsq3vQ', 'YEujDO49', '3SG49MKN', 'syfj72xE', 'LU1ds5P8', 
       'bR4NVhar', '5Sh31xmN', '7W9Jlwuv', 'c6kBh3nz', '4gPTaXJL', 'IVEd5SkM', 'ws5e0a8P', 
       've8BJ6s2', 'wzdFr0tP', 'TSCyFdqQ', 'FzAMfXcG', 'VheAb4kO', 'OUr5LnIC', '8otET1bP', 
       'oydxFsAD', 'xe3hz1UY', 'r1ZlYvpc', 'cm9EwQU2', 'AaukyJBU', '3j1upsIN', 'as7yltQJ', 
       'DUsu0zkH', 'u0Yz9rLJ', 'wbnUqQ2F', '3SJyhx2F', 'C3J8Oxnm', 'nsEwXu9k', 'MID0U1ZA', 
       'ykDaABeX', 'uz2f68hU', 'yc2EaZDJ', 'pNdxJnEf', 'Hv3UGPRz', 'Iw0ZfpKo', 'mUIVcqtR', 
       'm31I6cTD', 'Ow1Mhkaz', 'uSax3LOe', 'Ya50IKc9', '6LmHdbxa', 'PcH8M9RZ', '0i8TdrFw', 
       'rcutHBE9', 'uSVopLri', 'U96nNGkR', 'LkXRnEb6', '03FybMPz', 'Rdovw3C7', 'c2BnqCDT', 
       'KoMZHsqi', 'hFZCYHcM', 'yg4efXuG', 'BrMu6zV3', 'S7V4hIkY', 'vshEicrn', '4NGKAxBl', 
       'XEMtsYgn', 'XbLPHB76', '16bSkcB5', '1NnUGFwv', 'AG48trWL', 'a4b51Jfm', 'hyZK9XWP', 
       'aLBmP1ze', 'ipjQDwJf', 'YTjfkobL', 'Vdo50vyP', 'wjcFO1Nq', 'NhOsqela', 'LQomklIJ', 
       'KFUX4n0y', 'h8Rtvk3X', 'HYWLzKon', '3ef0KwqV', 'DpgEkYyX', '4U7SyvOx', 'etGDaAw7', 
       '8JhWU6ST', 'mjHPMUZ3', 'XTlaGDng', 'WU2zRqpH', 'SDwk2G1x', '6Ea5ckqf', '7Wy1hHl5']

In [11]:
# 读取数据集
# all_knowledge.csv
df_knowledge = pd.read_csv(os.path.join(data_path, 'all_knowledge.csv'))
        
# 对 section category knowledge_point 赋予课程之间唯一标示
df_knowledge['section'] = df_knowledge['course'] + df_knowledge['section']
df_knowledge['category'] = df_knowledge['course'] + df_knowledge['category']
df_knowledge['knowledge_point'] = df_knowledge['course'] + df_knowledge['knowledge_point']

# course.csv
df_course = pd.read_csv(os.path.join(data_path, 'course.csv'))

# student.csv
df_student = pd.read_csv(os.path.join(data_path, 'student.csv'))
    
# submission_s2.csv
df_test_score = pd.read_csv(os.path.join(data_path, 'submission_s2.csv'))
df_test_score.rename(columns={'pred': 'score'}, inplace=True)

# exam_score.csv
df_train_score = pd.read_csv(os.path.join(data_path, 'exam_score.csv'))#全量
#去除零异常值
df_train_score = df_train_score[df_train_score.score != 0]

df_train_score_jiajie = df_train_score[~df_train_score.exam_id.isin(ids)]#前三年
df_train_score = df_train_score[df_train_score.exam_id.isin(ids)]#最后一年



# course1_exams.csv ~ course8_exams.csv
df_list = []
df_exams_order_list = []
for i in range(1, 9):
    df_exams = pd.read_csv(os.path.join(data_path, 'course{}_exams.csv'.format(i)))
    df_exams_order = df_exams[['exam_id']]
    df_exams_order['order'] = df_exams.index
    df_exams_order_list.append(df_exams_order)

    del (df_exams_order)
    gc.collect()

    df_exams.set_index('exam_id', inplace=True)
    df_exams = df_exams.stack().to_frame()
    df_exams.reset_index(inplace=True)
    df_exams.rename(columns={
            'level_1': 'knowledge_point',
            0: 'ratio'
    },inplace=True)
    df_exams['knowledge_point'] = 'course{}'.format(i) + df_exams['knowledge_point']

    df_list.append(df_exams)

df_exams_ratio = pd.concat(df_list, axis=0)
df_exams_orders = pd.concat(df_exams_order_list, axis=0)

In [12]:
# 特征表
df_feature = pd.concat([df_train_score, df_test_score])#最后一年特征表
#前三年特征只需要统计train中的就行，不需要统计和test合并
df_train_score_jiajie_feature=df_train_score_jiajie.copy()

In [13]:
# ******* 学生特征 ******** #
# 成绩 max mean std cv
df_feature = stat(df_train_score, df_feature, ['student_id'], {'score': ['max', 'mean', 'std', 'median']})
df_feature = stat_cv(df_train_score, df_feature, ['student_id'], 'score')

#前三年
df_train_score_jiajie_feature = stat(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id'], {'score': ['max', 'mean', 'std', 'median']})
df_train_score_jiajie_feature = stat_cv(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id'], 'score')

In [14]:
# mean(个人考试成绩 / 本次考试平均成绩)
df_feature = stat_ratio_mean(df_train_score, df_feature, ['student_id'])

#前三年
df_train_score_jiajie_feature = stat_ratio_mean(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id'])

In [15]:
# 平均排名
df_feature = stat_rank(df_train_score, df_feature, ['student_id'])
#前三年
df_train_score_jiajie_feature = stat_rank(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id'])

In [16]:
# 性别
df_feature = df_feature.merge(df_student, on='student_id', how='left')

#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_student, on='student_id', how='left')

In [17]:
# score / 难度 衡量学生抗压能力
df_course_exam_feature = df_exams_ratio.merge(df_knowledge, on=['knowledge_point'], how='left')
df_course_exam_feature['complexity_ratio'] = df_course_exam_feature['ratio'] * df_course_exam_feature['complexity'] * 0.01


df_exam_complexity_sum = df_course_exam_feature.groupby(['exam_id'])['complexity_ratio'].sum().reset_index()
df_exam_complexity_sum.rename(columns={'complexity_ratio': 'exam_complexity'}, inplace=True)

df_score_complexity = df_train_score.merge(df_exam_complexity_sum, on='exam_id', how='left')
df_score_complexity_1 = df_train_score_jiajie.merge(df_exam_complexity_sum, on='exam_id', how='left')


df_score_complexity.drop_duplicates(inplace=True)
df_score_complexity_1.drop_duplicates(inplace=True)


df_score_complexity['s/c'] = df_score_complexity['score'] / df_score_complexity['exam_complexity']
df_score_complexity_1['s/c'] = df_score_complexity_1['score'] / df_score_complexity_1['exam_complexity']



df_feature = stat(df_score_complexity, df_feature, ['student_id'], {'s/c': ['mean', 'std', 'max', 'median']})
df_feature = stat(df_score_complexity, df_feature, ['student_id', 'course'], {'s/c': ['mean', 'std', 'max', 'median']})
df_feature = stat_cv(df_score_complexity, df_feature, ['student_id', 'course'], 's/c')
df_feature = stat_cv(df_score_complexity, df_feature, ['student_id'], 's/c')
#前三年
df_train_score_jiajie_feature = stat(df_score_complexity_1, df_train_score_jiajie_feature, ['student_id'], {'s/c': ['mean', 'std', 'max', 'median']})
df_train_score_jiajie_feature = stat(df_score_complexity_1, df_train_score_jiajie_feature, ['student_id', 'course'], {'s/c': ['mean', 'std', 'max', 'median']})
df_train_score_jiajie_feature = stat_cv(df_score_complexity_1, df_train_score_jiajie_feature, ['student_id', 'course'], 's/c')
df_train_score_jiajie_feature = stat_cv(df_score_complexity_1, df_train_score_jiajie_feature, ['student_id'], 's/c')




del (df_score_complexity)
del (df_score_complexity_1)
del (df_course_exam_feature)
del (df_exam_complexity_sum)
gc.collect()


In [18]:
# ******* 考试特征 ******** #
# 考试次序
df_feature = df_feature.merge(df_exams_orders, how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_exams_orders, how='left')


In [19]:
# 考试知识点范围（不为0个数/总个数）
df_temp = df_exams_ratio.groupby(['exam_id'])['ratio'].apply(lambda x: (x != 0).sum() / x.shape[
            0]).reset_index().rename(columns={'ratio': 'exam_kp_range'})
df_feature = df_feature.merge(df_temp, on='exam_id', how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_temp, on='exam_id', how='left')


del (df_temp)
gc.collect()

In [20]:
# 考试的知识点占比
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'knowledge_point', 'ratio']]
p = gen_feature(df_temp, 'exam_id', 'knowledge_point', 'ratio')
df_sparse_f = p.drop(['exam_id'], axis=1)
df_exam_f = p[['exam_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [ 'exam_point_ratio' + str(c) for c in df_no_sparse_f.columns]
df_exam_f = pd.concat([df_exam_f, df_no_sparse_f], axis=1)
df_feature = df_feature.merge(df_exam_f, on='exam_id', how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_exam_f, on='exam_id', how='left')

del (df_course_exam_feature)
del (df_temp)
del (df_sparse_f)
del (df_exam_f)
del (df_no_sparse_f)
del (p)
gc.collect()

df_feature.shape

In [21]:
# 考试各段落和种类占比
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'section', 'category', 'ratio']]

df_temp = stat(df_temp, df_temp, ['exam_id', 'section'], {'ratio': ['sum']})
df_temp = stat(df_temp, df_temp, ['exam_id', 'category'], {'ratio': ['sum']})

df_exam_f = df_temp[['exam_id']]
df_exam_f.drop_duplicates(inplace=True)
p = gen_feature(df_temp, 'exam_id', 'section', 'exam_id_section_ratio_sum')
df_exam_f = df_exam_f.merge(p, on='exam_id', how='left')
p = gen_feature(df_temp, 'exam_id', 'category', 'exam_id_category_ratio_sum')
df_exam_f = df_exam_f.merge(p, on='exam_id', how='left')

df_sparse_f = df_exam_f.drop(['exam_id'], axis=1)
df_exam_f = df_exam_f[['exam_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [
    'exam_section_cat_ratio' + str(c) for c in df_no_sparse_f.columns
]
df_exam_f = pd.concat([df_exam_f, df_no_sparse_f], axis=1)
df_feature = df_feature.merge(df_exam_f, on='exam_id', how='left')

#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_exam_f, on='exam_id', how='left')

del (df_course_exam_feature)
del (df_temp)
del (df_sparse_f)
del (df_exam_f)
del (df_no_sparse_f)
del (p)
gc.collect()


In [22]:
# 考试的各难度占比
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                                  on=['knowledge_point'],
                                                  how='left')
df_temp = df_course_exam_feature[['exam_id', 'complexity', 'ratio']]
df_temp = stat(df_temp, df_temp, ['exam_id', 'complexity'], {'ratio': ['sum']})
p = gen_feature(df_temp, 'exam_id', 'complexity', 'exam_id_complexity_ratio_sum')
df_feature = df_feature.merge(p, on='exam_id', how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(p, on='exam_id', how='left')

del (df_course_exam_feature)
del (df_temp)
del (p)
gc.collect()

df_feature.shape

In [23]:
# 考试总体难度
df_course_exam_feature = df_exams_ratio.merge(df_knowledge, on=['knowledge_point'], how='left')
df_course_exam_feature['complexity_ratio'] = df_course_exam_feature['ratio'] * df_course_exam_feature['complexity'] * 0.01
df_exam_complexity_sum = df_course_exam_feature.groupby(['exam_id'])['complexity_ratio'].sum().reset_index()
df_exam_complexity_sum.rename(columns={'complexity_ratio': 'exam_complexity'}, inplace=True)
df_feature = df_feature.merge(df_exam_complexity_sum, on='exam_id',  how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_exam_complexity_sum, on='exam_id',  how='left')

del (df_course_exam_feature)
del (df_exam_complexity_sum)
gc.collect()

df_feature.shape

In [26]:
# 针对考试知识点的有无构建句子，使用 w2v 为考试学习低维表示
from gensim.models.word2vec import Word2Vec 

df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'knowledge_point', 'ratio']]
df_temp = df_temp[df_temp.ratio != 0]

sentences = []
# 这边没去重， 句子会有重复
kps = df_temp['knowledge_point'].values
exam_ids = list(set(df_temp['exam_id'].values))

for kp in kps:
    df_kp = df_temp[df_temp.knowledge_point == kp]
    sentence = df_kp['exam_id'].values
    sentences.append(list(set(sentence)))

model= Word2Vec(size=dimension,window=2, min_count=1,iter=10)
model.build_vocab(sentences)
model.train(sentences, total_examples = model.corpus_count, epochs = model.iter)

embds = []
for exam_id in exam_ids:
    embd = model[exam_id]
    embds.append(embd)
df_temp = pd.DataFrame(embds)
df_temp.columns = ['w2v'+str(f) for f in df_temp.columns]
df_temp['exam_id'] = exam_ids

df_feature = df_feature.merge(df_temp, on='exam_id',  how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_temp, on='exam_id',  how='left')

del df_temp
gc.collect()

In [27]:
# ******* 课程特征 ******** #
# max mean std cv
df_feature = stat(df_train_score, df_feature, ['course'], {'score': ['mean', 'std', 'median']})
df_feature = stat_cv(df_train_score, df_feature, ['course'], 'score')

df_train_score_jiajie_feature = stat(df_train_score_jiajie, df_train_score_jiajie_feature, ['course'], {'score': ['mean', 'std', 'median']})
df_train_score_jiajie_feature = stat_cv(df_train_score_jiajie, df_train_score_jiajie_feature, ['course'], 'score')

In [28]:
# course_class 属性
df_feature = df_feature.merge(df_course, on='course', how='left')
#前三年
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_course, on='course', how='left')

In [29]:
# ******* 学生，考试组合特征 ******** #
# 学生在各个知识点上得分
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'knowledge_point', 'ratio']]
df_temp = df_temp[df_temp.ratio != 0]
df_temp = df_train_score.merge(df_temp, on='exam_id', how='left')




df_temp.drop(['exam_id', 'course'], axis=1, inplace=True)
df_temp['point_score'] = df_temp['ratio'] * df_temp['score'] * 0.01
g = df_temp.groupby(['student_id', 'knowledge_point'])['point_score'].agg({
    'student_point_score_mean':
    'mean',
})
g.reset_index(inplace=True)
p = gen_feature(g, 'student_id', 'knowledge_point', 'student_point_score_mean')
df_sparse_f = p.drop(['student_id'], axis=1)
df_student_f = p[['student_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [
    's_point_score' + str(c) for c in df_no_sparse_f.columns
]
df_student_f = pd.concat([df_student_f, df_no_sparse_f], axis=1)
df_feature = df_feature.merge(df_student_f, on='student_id', how='left')



del (df_course_exam_feature)
del (df_temp)
del (df_sparse_f)
del (df_student_f)
del (df_no_sparse_f)
del (g)
del (p)
gc.collect()

df_feature.shape

In [30]:
# ******* 学生，考试组合特征 ******** #
# 学生在各个知识点上得分
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'knowledge_point', 'ratio']]
df_temp = df_temp[df_temp.ratio != 0]
df_temp = df_train_score_jiajie.merge(df_temp, on='exam_id', how='left')




df_temp.drop(['exam_id', 'course'], axis=1, inplace=True)
df_temp['point_score'] = df_temp['ratio'] * df_temp['score'] * 0.01
g = df_temp.groupby(['student_id', 'knowledge_point'])['point_score'].agg({
    'student_point_score_mean':
    'mean',
})
g.reset_index(inplace=True)
p = gen_feature(g, 'student_id', 'knowledge_point', 'student_point_score_mean')
df_sparse_f = p.drop(['student_id'], axis=1)
df_student_f = p[['student_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [
    's_point_score' + str(c) for c in df_no_sparse_f.columns
]
df_student_f = pd.concat([df_student_f, df_no_sparse_f], axis=1)
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_student_f, on='student_id', how='left')



del (df_course_exam_feature)
del (df_temp)
del (df_sparse_f)
del (df_student_f)
del (df_no_sparse_f)
del (g)
del (p)
gc.collect()

df_train_score_jiajie_feature.shape

In [31]:
# 学生在各个section category上的平均得分（是否需要保留，后期测试）
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'section', 'category', 'ratio']]
df_temp = df_temp[df_temp.ratio != 0]

df_temp = stat(df_temp, df_temp, ['exam_id', 'section'], {'ratio': ['sum']})
df_temp = stat(df_temp, df_temp, ['exam_id', 'category'], {'ratio': ['sum']})

df_temp_section = df_temp[[
    'exam_id',
    'section',
    'exam_id_section_ratio_sum',
]]
df_temp_category = df_temp[[
    'exam_id', 'category', 'exam_id_category_ratio_sum'
]]

df_temp_section.drop_duplicates(inplace=True)
df_temp_category.drop_duplicates(inplace=True)

df_temp_section = df_train_score.merge(df_temp_section,
                                       on='exam_id',
                                       how='left')
df_temp_category = df_train_score.merge(df_temp_category,
                                        on='exam_id',
                                        how='left')

df_temp_section['section_score'] = df_temp_section[
    'exam_id_section_ratio_sum'] * df_temp_section['score'] * 0.01
df_temp_category['category_score'] = df_temp_category[
    'exam_id_category_ratio_sum'] * df_temp_category['score'] * 0.01

df_temp_section = stat(df_temp_section,
                       df_temp_section, ['student_id', 'section'],
                       agg={'section_score': ['max', 'mean', 'std', 'median']})
df_temp_category = stat(df_temp_category, df_temp_category,
                        ['student_id', 'category'],
                        {'category_score': ['max', 'mean', 'std', 'median']})

df_student_f = df_temp_section[['student_id']]
df_student_f.drop_duplicates(inplace=True)
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_mean')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_max')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_std')
df_student_f = df_student_f.merge(p, on='student_id', how='left')

p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_mean')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_max')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_std')
df_student_f = df_student_f.merge(p, on='student_id', how='left')

df_sparse_f = df_student_f.drop(['student_id'], axis=1)
df_student_f = df_student_f[['student_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [
    'student_section_cat_score' + str(c) for c in df_no_sparse_f.columns
]
df_student_f = pd.concat([df_student_f, df_no_sparse_f], axis=1)
df_feature = df_feature.merge(df_student_f, on='student_id', how='left')

del (df_temp)
del (df_course_exam_feature)
del (df_temp_section)
del (df_temp_category)
del (df_sparse_f)
del (df_no_sparse_f)
del (df_student_f)
gc.collect()

In [32]:
#前三年

# 学生在各个section category上的平均得分（是否需要保留，后期测试）
df_course_exam_feature = df_exams_ratio.merge(df_knowledge,
                                              on=['knowledge_point'],
                                              how='left')
df_temp = df_course_exam_feature[['exam_id', 'section', 'category', 'ratio']]
df_temp = df_temp[df_temp.ratio != 0]

df_temp = stat(df_temp, df_temp, ['exam_id', 'section'], {'ratio': ['sum']})
df_temp = stat(df_temp, df_temp, ['exam_id', 'category'], {'ratio': ['sum']})

df_temp_section = df_temp[[
    'exam_id',
    'section',
    'exam_id_section_ratio_sum',
]]
df_temp_category = df_temp[[
    'exam_id', 'category', 'exam_id_category_ratio_sum'
]]

df_temp_section.drop_duplicates(inplace=True)
df_temp_category.drop_duplicates(inplace=True)

df_temp_section = df_train_score_jiajie.merge(df_temp_section,
                                       on='exam_id',
                                       how='left')
df_temp_category = df_train_score_jiajie.merge(df_temp_category,
                                        on='exam_id',
                                        how='left')

df_temp_section['section_score'] = df_temp_section[
    'exam_id_section_ratio_sum'] * df_temp_section['score'] * 0.01
df_temp_category['category_score'] = df_temp_category[
    'exam_id_category_ratio_sum'] * df_temp_category['score'] * 0.01

df_temp_section = stat(df_temp_section,
                       df_temp_section, ['student_id', 'section'],
                       agg={'section_score': ['max', 'mean', 'std', 'median']})
df_temp_category = stat(df_temp_category, df_temp_category,
                        ['student_id', 'category'],
                        {'category_score': ['max', 'mean', 'std', 'median']})

df_student_f = df_temp_section[['student_id']]
df_student_f.drop_duplicates(inplace=True)
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_mean')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_max')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_section, 'student_id', 'section',
                'student_id_section_section_score_std')
df_student_f = df_student_f.merge(p, on='student_id', how='left')

p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_mean')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_max')
df_student_f = df_student_f.merge(p, on='student_id', how='left')
p = gen_feature(df_temp_category, 'student_id', 'category',
                'student_id_category_category_score_std')
df_student_f = df_student_f.merge(p, on='student_id', how='left')

df_sparse_f = df_student_f.drop(['student_id'], axis=1)
df_student_f = df_student_f[['student_id']]
pca = PCA(n_components=dimension, random_state=seed)
df_no_sparse_f = pd.DataFrame(pca.fit_transform(df_sparse_f))
df_no_sparse_f.columns = [
    'student_section_cat_score' + str(c) for c in df_no_sparse_f.columns
]
df_student_f = pd.concat([df_student_f, df_no_sparse_f], axis=1)
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_student_f, on='student_id', how='left')

del (df_temp)
del (df_course_exam_feature)
del (df_temp_section)
del (df_temp_category)
del (df_sparse_f)
del (df_no_sparse_f)
del (df_student_f)
gc.collect()

In [33]:
# ******* 学生，课程组合特征 ******** #
# max mean std cv
df_feature = stat(df_train_score, df_feature, ['student_id', 'course'],
                      {'score': ['max', 'mean', 'std', 'median']})
df_feature = stat_cv(df_train_score, df_feature, ['student_id', 'course'],
                         'score')


#前三年

df_train_score_jiajie_feature = stat(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id', 'course'],
                      {'score': ['max', 'mean', 'std', 'median']})
df_train_score_jiajie_feature = stat_cv(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id', 'course'],
                         'score')

In [34]:
# 课程平均排名
df_feature = stat_rank(df_train_score, df_feature, ['student_id', 'course'])
#前三年
df_train_score_jiajie_feature = stat_rank(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id', 'course'])

In [35]:
# 课程平均分
df_feature = stat_ratio_mean(df_train_score, df_feature, ['student_id', 'course'])
#前三年
df_train_score_jiajie_feature = stat_ratio_mean(df_train_score_jiajie, df_train_score_jiajie_feature, ['student_id', 'course'])

In [36]:
# 性别在课程考试 max mean std cv
df_feature = stat(df_feature, df_feature, ['gender', 'course'],
                      {'score': ['mean', 'std', 'median']})
df_feature = stat_cv(df_feature, df_feature, ['gender', 'course'], 'score')


#前三年
df_train_score_jiajie_feature = stat(df_train_score_jiajie_feature, df_train_score_jiajie_feature, ['gender', 'course'],
                      {'score': ['mean', 'std', 'median']})
df_train_score_jiajie_feature = stat_cv(df_train_score_jiajie_feature, df_train_score_jiajie_feature, ['gender', 'course'], 'score')

In [37]:
# 学生课程最近三次考试平均成绩
df_score = pd.concat([df_train_score, df_test_score])
df_temp = df_score.merge(df_exams_orders, how='left')
df_temp.sort_values(['student_id', 'course', 'order'], inplace=True)
df_temp['pre_3_score_mean'] = df_temp.groupby(
        ['student_id',
         'course'])['score'].shift(1).rolling(window=3, min_periods=1).mean()
# df_temp['pre_3_score_std'] = df_temp.groupby(
#         ['student_id',
#          'course'])['score'].shift(1).rolling(window=3, min_periods=1).std()
# df_temp['pre_3_score_diff_mean'] = df_temp.groupby(
#         ['student_id',
#          'course'])['score'].shift(1).diff().rolling(window=3, min_periods=1).mean()
df_temp = df_temp[['student_id', 'course', 'exam_id', 'pre_3_score_mean']]
df_feature = df_feature.merge(df_temp,
                                   on=['student_id', 'course', 'exam_id'],
                                   how='left')
del (df_score)
del (df_temp)
gc.collect()

In [38]:
#前三年
# 学生课程最近三次考试平均成绩
# df_score = pd.concat([df_train_score, df_test_score])
df_temp = df_train_score_jiajie.merge(df_exams_orders, how='left')
df_temp.sort_values(['student_id', 'course', 'order'], inplace=True)
df_temp['pre_3_score_mean'] = df_temp.groupby(
        ['student_id',
         'course'])['score'].shift(1).rolling(window=3, min_periods=1).mean()
df_temp = df_temp[['student_id', 'course', 'exam_id', 'pre_3_score_mean']]
df_train_score_jiajie_feature = df_train_score_jiajie_feature.merge(df_temp,
                                   on=['student_id', 'course', 'exam_id'],
                                   how='left')
del (df_temp)
gc.collect()

In [39]:
# LabelEncoder
for f in ['course', 'course_class']:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))
    df_train_score_jiajie_feature[f] = lbl.fit_transform(df_train_score_jiajie_feature[f].astype(str))

In [40]:
#在这里应该合并df_feature 和df_train_score_jiajie_feature
len_dfeature=df_feature.shape[0]
feature=df_feature.append(df_train_score_jiajie_feature).reset_index(drop=True)

In [41]:
feature = select_feature(feature)

In [42]:
df_feature=feature[:len_dfeature]#最后一年
df_jiajie=feature[len_dfeature:]#前三年

In [43]:
#构造嫁接学习model，然后用model取预测最后一年的分数作为最后一模型的feature

import matplotlib.pyplot as plt

seed = 2008
nfold = 5

ycol = 'score'

feature_names = list(
    filter(lambda x: x not in [ycol, 'student_id','exam_id'], df_jiajie.columns))

jiajie_pred = np.zeros((df_feature.shape[0],))
kfolder = KFold(n_splits=nfold, shuffle=True, random_state=seed)
df_val_list = []
score_train_total = 0
score_val_total = 0
for fold_id, (trn_idx, val_idx) in enumerate(kfolder.split(df_jiajie)):
    print(
        '\nxgboost Fold_{} Training ================================\n'.format(
            fold_id+1))
    X_train = df_jiajie.iloc[trn_idx][feature_names]
    Y_train = df_jiajie.iloc[trn_idx][ycol]

    X_val = df_jiajie.iloc[val_idx][feature_names]
    Y_val = df_jiajie.iloc[val_idx][ycol]

    jiajie_model = xgb.XGBRegressor(random_state=seed,
                                 max_depth=8,
                                 n_estimators=10000,
                                 min_child_weight=300,
                                 colsample_bytree=0.8,
                                 subsample=0.8,
                                 learning_rate=0.1,
                                 reg_alpha=1,
                                 reg_lambda=0.8,
                                 objective="reg:linear",
                                 eta=0.1)

    jiajie_model.fit(X_train,
                  Y_train,
                  eval_set=[(X_train, Y_train), (X_val, Y_val)],
                  verbose=1000,
                  early_stopping_rounds=500,
                  eval_metric='rmse')


    jiajie_pred += jiajie_model.predict(df_feature[feature_names]) / nfold

In [44]:
df_feature['jiajie_pred_score']=jiajie_pred

In [45]:
df_feature[['jiajie_pred_score', 'exam_id','student_id']].to_csv(os.path.join(current_path, 'jiajie.csv'),index=False)

In [ ]:
df_feature.head()